In [48]:
import pandas as pd

In [49]:
flight_data = pd.read_csv("data/datasets/flight-data.csv")
passenger_data = pd.read_csv("data/datasets/passenger-data.csv")

In [50]:
# creating the airport table
# IATACode VARCHAR(3) PRIMARY KEY,
# Name VARCHAR(100) NOT NULL,
# City VARCHAR(50) NOT NULL,
# Country VARCHAR(50) NOT NULL

airport_features = ["IATACode", "City", "State"]
airport = pd.DataFrame(columns=airport_features)

In [51]:
iata = pd.DataFrame(
    {
        "IATACode": flight_data["ORIGIN"].append(flight_data["DEST"]),
        "City": flight_data["ORIGIN_CITY"].append(flight_data["DEST_CITY"]),
    }
).drop_duplicates()

airport[["City", "State"]] = iata["City"].str.split(",", expand=True)
airport["IATACode"] = iata["IATACode"]
airport

,IATACode,City,State
0,FLL,Fort Lauderdale,FL
1,MSP,Minneapolis,MN
2,DEN,Denver,CO
4,MCO,Orlando,FL
5,DAL,Dallas,TX
...,...,...,...
101833,OGD,Ogden,UT
131177,STC,St. Cloud,MN
142595,UIN,Quincy,IL
169922,GST,Gustavus,AK


In [52]:
# Flight table
# Date DATETIME,                - FL_DATE
#     FlightNumber VARCHAR(10), - FL_NUMBER
#     AirlineName VARCHAR(20),  - AIRLINE
#     DeptAirportID VARCHAR(3), - ORIGIN
#     ArrvAirportID VARCHAR(3), - DEST
#     DeptTime DATETIME,        - CRS_DEP_TIME
#     ArrvTime DATETIME,        - CRS_ARR_TIME
#     ActualDeptTime DATETIME,  - DEP_TIME
#     AcutalArrvTime DATETIME,  - ARR_TIME

flight_features = [
    "Date",
    "FlightNumber",
    "AirlineName",
    "DeptAirportID",
    "ArrvAirportID",
    "DeptTime",
    "ArrvTime",
    "ActualDeptTime",
    "AcutalArrvTime",
]
flight = pd.DataFrame(columns=flight_features)

flight["Date"] = flight_data["FL_DATE"]
flight["FlightNumber"] = (
     flight_data["AIRLINE_CODE"] + " " + flight_data["FL_NUMBER"].astype(str)
)
flight["AirlineName"] = flight_data["AIRLINE"]
flight["DeptAirportID"] = flight_data["ORIGIN"]
flight["ArrvAirportID"] = flight_data["DEST"]
flight["DeptTime"] = flight_data["CRS_DEP_TIME"]
flight["ArrvTime"] = flight_data["CRS_ARR_TIME"]
flight["ActualDeptTime"] = flight_data["DEP_TIME"]
flight["AcutalArrvTime"] = flight_data["ARR_TIME"]
flight.head(20)

,Date,FlightNumber,AirlineName,DeptAirportID,ArrvAirportID,DeptTime,ArrvTime,ActualDeptTime,AcutalArrvTime
0,2019-01-09,UA 1562,United Air Lines Inc.,FLL,EWR,1155,1501,1151.0,1447.0
1,2022-11-19,DL 1149,Delta Air Lines Inc.,MSP,SEA,2120,2315,2114.0,2310.0
2,2022-07-22,UA 459,United Air Lines Inc.,DEN,MSP,954,1252,1000.0,1252.0
3,2023-03-06,DL 2295,Delta Air Lines Inc.,MSP,SFO,1609,1829,1608.0,1853.0
4,2020-02-23,NK 407,Spirit Air Lines,MCO,DFW,1840,2041,1838.0,2040.0
5,2019-07-31,WN 665,Southwest Airlines Co.,DAL,OKC,1010,1110,1237.0,1331.0
6,2023-06-11,AA 2134,American Airlines Inc.,DCA,BOS,1010,1159,1001.0,1130.0
7,2019-07-08,YX 4464,Republic Airline,HSV,DCA,1643,1945,1637.0,2008.0
8,2023-02-12,NK 590,Spirit Air Lines,IAH,LAX,530,717,527.0,706.0
9,2020-08-22,AS 223,Alaska Airlines Inc.,SEA,FAI,2125,2355,2116.0,2356.0


In [53]:
# Status

# Date DATETIME,                        - FL_DATE
# FlightNumber VARCHAR(10),             - FL_NUMBER
# FlightStatus VARCHAR(20) NOT NULL,    - ON_TIME   if DEP_DELAY <= 0
#                                       - DELAYED   if DEP_DELAY > 0
#                                       - CANCELLED if CANCELLED == 1.0

status_features = ["Date", "DeptTime", "FlightNumber", "FlightStatus"]


def get_flight_status(row):
    if row["CANCELLED"] == 1.0:
        return "CANCELLED"
    elif row["DEP_DELAY"] > 0:
        return "DELAYED"
    else:
        return "ON TIME"


status = pd.DataFrame(columns=status_features)
status["Date"] = flight_data["FL_DATE"]
status.loc[:, "DeptTime"] = flight_data["CRS_DEP_TIME"]
status["FlightNumber"] = (
    flight_data["AIRLINE_CODE"] + " " + flight_data["FL_NUMBER"].astype(str)
)
status["FlightStatus"] = flight_data.apply(get_flight_status, axis=1)

In [54]:
status.head(10)

,Date,DeptTime,FlightNumber,FlightStatus
0,2019-01-09,1155,UA 1562,ON TIME
1,2022-11-19,2120,DL 1149,ON TIME
2,2022-07-22,954,UA 459,DELAYED
3,2023-03-06,1609,DL 2295,ON TIME
4,2020-02-23,1840,NK 407,ON TIME
5,2019-07-31,1010,WN 665,DELAYED
6,2023-06-11,1010,AA 2134,ON TIME
7,2019-07-08,1643,YX 4464,ON TIME
8,2023-02-12,530,NK 590,ON TIME
9,2020-08-22,2125,AS 223,ON TIME


In [60]:
import csv
from datetime import datetime

INPUT_FILE = "data/datasets/passenger-data.csv"
OUTPUT_FILE = "data/datasets/clean-passenger-data.csv"
# Validates time format HH:MM:SS
def is_valid_time(time_str):
    try:
        datetime.strptime(time_str, "%H:%M:%S")
        return True
    except ValueError:
        return False


# Read, validate, and write to new CSV
def filter_invalid_times(input_file, output_file):
    with open(input_file, mode="r", newline="", encoding="utf-8") as infile, open(
        output_file, mode="w", newline="", encoding="utf-8"
    ) as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()

        for row in reader:
            if is_valid_time(row.get("departure_time", "")):
                writer.writerow(row)


filter_invalid_times(INPUT_FILE, OUTPUT_FILE)
print("Filtering complete. Invalid times removed.") 

Filtering complete. Invalid times removed.


In [61]:
# passenger

# Email VARCHAR(100) PRIMARY KEY,
# FirstName VARCHAR(50) NOT NULL,
# LastName VARCHAR(50) NOT NULL

passenger_data = pd.read_csv("data/datasets/clean-passenger-data.csv")
passenger_features = ["Email", "FirstName", "LastName"]
passenger = pd.DataFrame(columns=passenger_features)

passenger["Email"] = passenger_data["email"]
passenger["FirstName"] = passenger_data["first_name"]
passenger["LastName"] = passenger_data["last_name"]

In [62]:
passenger.head()

,Email,FirstName,LastName
0,Andrew479Wood@gmail.com,Andrew,Wood
1,Brooklyn282Lee@gmail.com,Brooklyn,Lee
2,Cheryl369Gutierrez@gmail.com,Cheryl,Gutierrez
3,Laura484Wilson@gmail.com,Laura,Wilson
4,Janice222Hall@gmail.com,Janice,Hall


In [63]:
passenger_data.head()

,passenger_id,first_name,last_name,email,booking_id,flight_id,booking_date,seat_number,booking_status,departure_time
0,1,Andrew,Wood,Andrew479Wood@gmail.com,1,AA_1776_2023-04-14_1,2022-4-14,V21,CONFIRMED,13:00:00
1,2,Brooklyn,Lee,Brooklyn282Lee@gmail.com,2,YX_4942_2021-09-21_2,2020-9-21,R6,CONFIRMED,10:19:00
2,3,Cheryl,Gutierrez,Cheryl369Gutierrez@gmail.com,3,WN_883_2019-04-30_3,2018-4-30,J16,CONFIRMED,20:00:00
3,4,Laura,Wilson,Laura484Wilson@gmail.com,4,OO_3370_2020-07-18_4,2019-7-18,U14,CONFIRMED,17:50:00
4,5,Janice,Hall,Janice222Hall@gmail.com,5,AA_2345_2021-12-28_5,2020-12-28,P10,CONFIRMED,14:45:00


In [64]:
# CREATE TABLE Booking (
#     BookingId INT PRIMARY KEY AUTO_INCREMENT,
#     FlightDate DATE,
#     FlightNumber VARCHAR(10),
#     Email VARCHAR(100),
#     BookingDate DATE NOT NULL,
#     Status VARCHAR(20) NOT NULL,
#     FOREIGN KEY (FlightDate, FlightDeptTime, FlightNumber) REFERENCES Flight(Date, DeptTime, AirlineCode, FlightNumber),
#     FOREIGN KEY (Email) REFERENCES Passenger(Email)
# );

booking_features = [
    "BookingId",
    "FlightDate",
    "DeptTime",
    "FlightNumber",
    "Email",
    "BookingDate",
    "Status",
]


def convertTime(series):
    return series.str.split(":").apply(lambda x: int(x[0]) * 100 + int(x[1]))


booking = pd.DataFrame(columns=booking_features)
booking["BookingId"] = passenger_data["booking_id"]
booking["FlightDate"] = passenger_data["flight_id"].str.split("_", expand=True)[2]
booking["DeptTime"] = convertTime(passenger_data["departure_time"])
booking["FlightNumber"] = (
    passenger_data["flight_id"].str.split("_", expand=True)[0]  
    + " "
    + passenger_data["flight_id"].str.split("_", expand=True)[1]
)
booking["Email"] = passenger_data["email"]
booking["BookingDate"] = passenger_data["booking_date"]
booking["Status"] = passenger_data["booking_status"]
booking

,BookingId,FlightDate,DeptTime,FlightNumber,Email,BookingDate,Status
0,1,2023-04-14,1300,AA 1776,Andrew479Wood@gmail.com,2022-4-14,CONFIRMED
1,2,2021-09-21,1019,YX 4942,Brooklyn282Lee@gmail.com,2020-9-21,CONFIRMED
2,3,2019-04-30,2000,WN 883,Cheryl369Gutierrez@gmail.com,2018-4-30,CONFIRMED
3,4,2020-07-18,1750,OO 3370,Laura484Wilson@gmail.com,2019-7-18,CONFIRMED
4,5,2021-12-28,1445,AA 2345,Janice222Hall@gmail.com,2020-12-28,CONFIRMED
...,...,...,...,...,...,...,...
713,1014,2020-02-04,1737,OO 3694,Dorothy669Mitchell@gmail.com,2019-2-4,CONFIRMED
714,1016,2023-08-08,2219,UA 352,Oliver287Patel@gmail.com,2022-8-8,CONFIRMED
715,1017,2021-10-25,1730,B6 529,Lillian727Martinez@gmail.com,2020-10-25,CONFIRMED
716,1019,2020-09-28,1220,WN 5751,Cynthia115Phillips@gmail.com,2019-9-28,CONFIRMED


In [65]:
passenger_data['departure_time']

0      13:00:00
1      10:19:00
2      20:00:00
3      17:50:00
4      14:45:00
         ...   
713    17:37:00
714    22:19:00
715    17:30:00
716    12:20:00
717    11:33:00
Name: departure_time, Length: 718, dtype: object

In [66]:
airport.to_csv("data/tables/airport.csv", index=False)
flight.to_csv("data/tables/flight.csv", index=False)
status.to_csv("data/tables/status.csv", index=False)
passenger.to_csv("data/tables/passenger.csv", index=False)
booking.to_csv("data/tables/booking.csv", index=False)